#

#

#

#

#

#

# 함수 데커레이터와 클로저

## 데커레이터 기본 지식

### 일반적으로 데커레이터는 함수를 다른 함수로 대체한다

In [5]:
def deco(func):
    def inner():
        print('running inner()')
    return inner # deco()가 inner()함수 객체를 반환한다.

In [6]:
@deco
def target():  # target()을 deco로 데커레이트했다.
    print('running target()')

In [7]:
target() # 데커레이트된 target()을 호출하면 실제로는 inner()를 실행한다.

running inner()


In [8]:
target #조사해보면 target이 inner()를 가리키고 있음을 알 수 있다.

<function __main__.deco.<locals>.inner()>

## 파이썬이 데커레이터를 실행하는 시점

### 예제 7-2 registration.py 모듈

In [10]:
registry = []

def register(func):
    print('running register(%s)' % func)
    registry.append(func)
    return func

@register
def f1():
    print('running f1()')
    
@register
def f2():
    print('running f2()')

def f3():
    print('running f3()')

def main():
    print('running main()')
    print('registry ->',registry)
    f1()
    f2()
    f3()
    
if __name__=='__main__':
    main()

running register(<function f1 at 0x0000020DDD4DB9C0>)
running register(<function f2 at 0x0000020DDD4D82C0>)
running main()
registry -> [<function f1 at 0x0000020DDD4DB9C0>, <function f2 at 0x0000020DDD4D82C0>]
running f1()
running f2()
running f3()


## 데커레이터로 개선한 전략 패턴

### 예제 7-3 promotion 데커레이터로 채운 promos 리스트

In [17]:
promos = []

def promotion(promo_func):
    promos.append(promo_func)
    return promo_func

@promotion
def fidelity(order):
    """충성도 포인트가 1000점 이상인 고객에게 전체 5% 할인 적용"""
    return order.total() * .05 if order.customer.fidelity >= 1000 else 0

@promotion
def bulk_item(order):
    """20개 이상의 동일 상품을 구입하면 10% 할인 적용"""
    discount = 0
    for item in order.cart:
        if item.quantity >= 20:
            discount += item.total() *.1
    return discount

@promotion
def large_order(order):
    """10종류 이상의 상품을 구입하면 전체 7% 할인 적용"""
    distinct_items = {item.product for item in order.cart}
    if len(discount_items) >= 10:
        return order.total() * .07
    return 0

def best_promo(order):
    """최대로 할인 받을 금액을 반환한다"""
    return max(promo(order) for promo in promos)

In [18]:
# 255p

## 변수 범위 규칙


In [1]:
def f1(a):
    print(a)
    print(b)

In [2]:
f1(3) ## 에러남

3


NameError: name 'b' is not defined

In [3]:
b=6
f1(3) #에러가 안난다 

3
6


In [25]:
dis(f1)

  1           0 RESUME                   0

  2           2 LOAD_GLOBAL              1 (NULL + print)
             14 LOAD_FAST                0 (a)
             16 PRECALL                  1
             20 CALL                     1
             30 POP_TOP

  3          32 LOAD_GLOBAL              1 (NULL + print)
             44 LOAD_GLOBAL              2 (b)
             56 PRECALL                  1
             60 CALL                     1
             70 POP_TOP
             72 LOAD_CONST               0 (None)
             74 RETURN_VALUE


In [6]:
b=6
def f2(a):
    print(a)
    print(b)
    b=9

In [7]:
f2(3)

3


UnboundLocalError: cannot access local variable 'b' where it is not associated with a value

In [19]:
b=6
def f3(a):
    global b
    print(a)
    print(b)
    b=11

In [20]:
f3(3)

3
6


In [21]:
b

11

In [22]:
f3(3)

3
11


In [18]:
b=30
b

30

In [24]:
from dis import dis
dis(f1)

  1           0 RESUME                   0

  2           2 LOAD_GLOBAL              1 (NULL + print)
             14 LOAD_FAST                0 (a)
             16 PRECALL                  1
             20 CALL                     1
             30 POP_TOP

  3          32 LOAD_GLOBAL              1 (NULL + print)
             44 LOAD_GLOBAL              2 (b)
             56 PRECALL                  1
             60 CALL                     1
             70 POP_TOP
             72 LOAD_CONST               0 (None)
             74 RETURN_VALUE


In [26]:
dis(f2)

  2           0 RESUME                   0

  3           2 LOAD_GLOBAL              1 (NULL + print)
             14 LOAD_FAST                0 (a)
             16 PRECALL                  1
             20 CALL                     1
             30 POP_TOP

  4          32 LOAD_GLOBAL              1 (NULL + print)
             44 LOAD_FAST                1 (b)
             46 PRECALL                  1
             50 CALL                     1
             60 POP_TOP

  5          62 LOAD_CONST               1 (9)
             64 STORE_FAST               1 (b)
             66 LOAD_CONST               0 (None)
             68 RETURN_VALUE


## 클로저

In [39]:
class Averager():
    
    def __init__(self):
        self.series = []
        
    def __call__(self, new_value):
        self.series.append(new_value)
        total= sum(self.series)
        return total/len(self.series)

In [32]:
avg = Averager()

In [33]:
avg(10)

10.0

In [34]:
avg(11)

10.5

In [35]:
avg(12)

11.0

In [38]:
def make_averager(): # 이동 평균을 계산하는 고위 함수 
    series = []
    
    def averager(new_value):
        series.append(new_value)
        total = sum(series)
        return total/len(series)
    return averager

In [37]:
# avg. 262p에 왜 code가 없지 
avg.__clo ## 클로저도 없는데? 

AttributeError: 'Averager' object has no attribute '__code__'

In [ ]:
# nonlocal -> 변수를 논로컬로 선언하면 함수 안에서 변수에 새로운 값을 할당하더라도 그 변수는 자유변수임을
# 나타낸다. 

In [ ]:
# 전체 이력을 유지하지 않고 이동 평균을 계산하는 잘못된 고위 함수

In [40]:
def make_averager():
    count =0
    total =0
    
    def averager(new_value):
        count +=1 
        total += new_value
        return total/count
    
    return averager

In [42]:
avg = make_averager()
avg(10)  # 에러나는거 맞음 

UnboundLocalError: cannot access local variable 'count' where it is not associated with a value

In [ ]:
# 전체 이력을 유지하지 않고 이동 평균 계산하기 ( nonlocal로 수정)

## nonlocal 선언

In [43]:
def make_averager():
    count =0
    total =0
    
    def averager(new_value):
        nonlocal count, total
        count += 1
        total += new_value
        return total/ count
    
    
    return averager

## 간단한 데커레이터 구현하기

In [ ]:
# 265p

In [1]:
import time

In [2]:
def clock(func):
    def clocked(*args): #1 
        t0 = time.perf_counter()
        result =func(*args) #2
        elapsed = func.__name__
        arg_str = ', '.join(repr(arg) for arg in args)
        print('[%0.8fs] %s(%s) -> %r'% (elapsed, name, arg_str, result))
        return result
    return clocked #3

1. 내부 함수 clocked()가 임의 개수의 위치 인수를 받을 수 있도록 정의한다.
2. clocked()에 대한 클로저에 자유 변수가 func 가 들어가야 이 코드가 작동한다.
3. 내부 함수를 반환해서 데커레이트된 함수를 대체한다.

In [5]:
# clockdeco_demo.py

import time
from clockdeco import clock 

@clock
def snooze(seconds):
    time.sleep(seconds)
    
@clock
def factorial(n):
    return 1 if n < 2 else n*factorial(n-1)

if __name__=='__main__':
    print('*' * 40, 'Calling snooze(.123)')
    snooze(.123)
    print('*' * 40, 'Calling factorial(6)')
    print('6! =',factorial(6))

ModuleNotFoundError: No module named 'clockdeco'

 위에거 왜 작동을 안하는지 나중에 확인 

In [7]:
# clockdeco2.py

import time
import functools

def clock(func):
    @functools.wraps(func)
    def clocked(*args, **kwargs):
        t0: time.time()
        result = func(*args, **kwargs)
        elapse = time.time() - t0
        name = func.__name__
        arg_lst = []
        if args:
            arg_lst.append(', '.join(repr(arg) for arg in args))
        if kwargs:
            pairs = ['%s=%r' % (k,w) for k, w in sorted (kwargs.items())]
            arg_lst.append(', '.join(pairs))
        arg_str = ', '.join(arg_lst)
        print('[%0.8fs] %s(%s) -> %r ' % (elapsed, name, arg_str, result))
        return result 
    return clocked

## 표준 라이브러리에서 제공하는 데커레이터

### functools.lru_cache()를 이용한 메모이제이션

#### 예제7-18 fibo_demo.py 피보나치 수열에서 n번째 숫자를 아주 값비싸게 계산하는 방식

In [9]:
from clockdeco import clock

@clock
def fibonacci(n):
    if n<2:
        return n
    return fibonacci(n-2) + fibonacci(n-1)

if __name__=='__main__':
    print(fibonacci(6))

ModuleNotFoundError: No module named 'clockdeco'

In [ ]:
# 아니 왜 clockdeco가 없다는거냐 

In [ ]:
#270 